In [34]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from sklearn.model_selection import TimeSeriesSplit

import random

import os
import json

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner
from datetime import datetime

from model import TCNRegressor
from tuning import ray_train

import train
import utils
import tuning
import data
from data import transform_data 
from data import TimeSeriesDataset


if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [35]:
# Number of trials for tuning
trials = 100

# Fixed parameters (not tuned)
fixed_params = {
    "lag_periods": [1, 3, 12],
    "features": ['BIR', 'BOC', 'Other Offices', 'TotalTrade_PHPMN', 'NominalGDP_disagg', 'Pop_disagg'],
    "labels": ["BIR", 'BOC', 'Other Offices'],
    "dummy_vars": ['COVID-19', 'TRAIN', 'CREATE', 'FIST', 'BIR_COMM'],
    "experiment_name": "test"
}

# Resource detection (equivalent to your resources setup)
if torch.cuda.is_available():
    device = "cuda"
    n_jobs = 4  # Run 4 trials concurrently (equivalent to max_concurrent_trials=4)
    print("Using CUDA GPU")
elif torch.backends.mps.is_available():
    device = "mps"
    n_jobs = 1
    print("Using MPS (Mac GPU) - GPU allocation not specified")
else:
    device = "cpu"
    n_jobs = 4
    print("Using CPU only")

def objective(trial):
    """
    Optuna objective function - equivalent to ray_train_with_resources
    Replace the train_model call with your actual ray_train function
    """
    
    # Define search space (same as your Ray Tune search_space)
    config = {
        # === MODEL ARCHITECTURE ===
            # === MODEL ARCHITECTURE ===
        "num_channels": trial.suggest_categorical("num_channels", [[64], [64, 64], [64, 64, 64], [128], [128, 128], [128, 128, 128], [256], [256, 256], [256, 256, 256]]),
        "kernel_size": trial.suggest_categorical("kernel_size", [2, 3, 4, 5]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.4),
        "output_size": 3,  # Fixed: number of output variables (BIR, BOC, Other Offices)
        
        # === OPTIMIZER ===
        "lr": trial.suggest_float("lr", 5e-4, 5e-2, log=True),
        "wd": trial.suggest_float("wd", 1e-6, 1e-4, log=True),
        
        # === SCHEDULER ===
        "factor": trial.suggest_float("factor", 0.2, 0.5),
        "patience": trial.suggest_categorical("patience", [5, 10]),
        
        # === TRAINING ===
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),
        "seq_len": trial.suggest_categorical("seq_len", [3, 6, 12, 18, 24]),
        "l1_lambda": trial.suggest_float("l1_lambda", 1e-5, 1e-2, log=True),
        
        # === DEVICE ===
        "device": device,
    }
    
    # Add fixed parameters
    config.update(fixed_params)
    
    try:
        # REPLACE THIS with your actual ray_train function call
        # Your ray_train should accept config and optionally trial for intermediate reporting
        result = ray_train(config, trial=trial)
        
        # Handle result (loss, std) or just loss
        if isinstance(result, tuple):
            loss, std = result
            trial.set_user_attr("std", std)
        else:
            loss = result
            trial.set_user_attr("std", None)
        
        return loss
    
    except optuna.TrialPruned:
        raise
    except Exception as e:
        print(f"Trial {trial.number} failed: {str(e)}")
        return float('inf')

# ASHA Scheduler equivalent (same parameters as your ASHAScheduler)
# max_t=50, grace_period=10, reduction_factor=2
pruner = SuccessiveHalvingPruner(
    min_resource=10,        # grace_period
    reduction_factor=2,     # reduction_factor
    min_early_stopping_rate=0
)

# Optuna Search (TPE sampler - same as OptunaSearch)
sampler = TPESampler(
    seed=42,
    n_startup_trials=10,
    multivariate=True
)

# Create study (equivalent to creating Tuner)
study = optuna.create_study(
    direction="minimize",  # mode="min"
    sampler=sampler,
    pruner=pruner,
    study_name="gru_tuning"
)

# Progress reporter (equivalent to CLIReporter)
class ProgressReporter:
    def __init__(self, max_trials, metric_columns=None):
        self.max_trials = max_trials
        self.metric_columns = metric_columns or ["loss", "std"]
        self.trials_completed = 0
        self.trials_running = 0
        self.best_loss = float('inf')
        self.recent_trials = []
        
    def __call__(self, study, trial):
        self.trials_completed += 1
        
        # Update best loss
        if study.best_value < self.best_loss:
            self.best_loss = study.best_value
        
        # Get trial info
        state = trial.state.name
        loss = trial.value if trial.value is not None else float('inf')
        std = trial.user_attrs.get("std", None)
        
        # Store recent trial info
        trial_info = {
            'trial_number': trial.number,
            'state': state,
            'loss': loss,
            'std': std,
            'params': trial.params
        }
        self.recent_trials.append(trial_info)
        
        # Keep only last 15 trials (max_progress_rows=15)
        if len(self.recent_trials) > 15:
            self.recent_trials.pop(0)
        
        # Print intermediate table (similar to CLIReporter)
        if self.trials_completed % 5 == 0 or self.trials_completed == 1:
            self._print_table()
    
    def _print_table(self):
        print("\n" + "="*100)
        print(f"Progress: {self.trials_completed}/{self.max_trials} trials | Best loss: {self.best_loss:.4f}")
        print("="*100)
        
        # Header
        print(f"{'Trial':<8} {'State':<12} {'Loss':<12} {'Std':<12} {'hidden_size':<12} {'lr':<12} {'batch_size':<12}")
        print("-"*100)
        
        # Sorted by loss (sort_by_metric=True)
        sorted_trials = sorted(
            [t for t in self.recent_trials if t['loss'] != float('inf')],
            key=lambda x: x['loss']
        )
        
        for t in sorted_trials[:15]:  # max_progress_rows=15
            loss_str = f"{t['loss']:.4f}" if t['loss'] != float('inf') else "N/A"
            std_str = f"{t['std']:.4f}" if t['std'] is not None else "N/A"
            hidden_size = t['params'].get('hidden_size', 'N/A')
            lr = f"{t['params'].get('lr', 0):.2e}" if 'lr' in t['params'] else "N/A"
            batch_size = t['params'].get('batch_size', 'N/A')
            
            print(f"{t['trial_number']:<8} {t['state']:<12} {loss_str:<12} {std_str:<12} {hidden_size:<12} {lr:<12} {batch_size:<12}")
        
        print("="*100 + "\n")

# Statistics tracker (prints every 30 seconds worth of trials)
class StatsTracker:
    def __init__(self):
        self.start_time = datetime.now()
        self.last_report_time = self.start_time
        
    def __call__(self, study, trial):
        current_time = datetime.now()
        elapsed_since_report = (current_time - self.last_report_time).total_seconds()
        
        # Report every ~30 seconds (max_report_frequency=30)
        if elapsed_since_report >= 30:
            self.last_report_time = current_time
            
            completed = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
            pruned = len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])
            failed = len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])
            
            total_elapsed = (current_time - self.start_time).total_seconds()
            
            print(f"\n{'#'*100}")
            print(f"STATUS UPDATE (after {total_elapsed/60:.1f} minutes)")
            print(f"{'#'*100}")
            print(f"Completed: {completed} | Pruned: {pruned} | Failed: {failed}")
            if completed > 0:
                print(f"Best loss so far: {study.best_value:.4f}")
                print(f"Average time per trial: {total_elapsed/trial.number:.1f}s")
            print(f"{'#'*100}\n")

# Initialize reporters
reporter = ProgressReporter(max_trials=trials, metric_columns=["loss", "std"])
stats_tracker = StatsTracker()

# Run optimization (equivalent to tuner.fit())
print("="*100)
print("STARTING HYPERPARAMETER TUNING")
print("="*100)
print(f"Total trials: {trials}")
print(f"Concurrent trials: {n_jobs}")
print(f"Device: {device}")
print(f"Study name: gru_tuning")
print(f"Storage: ray_results/ (Optuna equivalent)")
print("="*100 + "\n")

# Create results directory (equivalent to storage_path)
results_dir = os.path.abspath("ray_results/gru_tuning")
os.makedirs(results_dir, exist_ok=True)

# Optimize
study.optimize(
    objective,
    n_trials=trials,
    callbacks=[reporter, stats_tracker],
    n_jobs=n_jobs,  # max_concurrent_trials=4
    show_progress_bar=True,
    gc_after_trial=True
)

# Get best result (equivalent to results.get_best_result)
best_trial = study.best_trial
best_config = best_trial.params.copy()
best_config.update(fixed_params)  # Add fixed params

# Print results (same format as your original code)
print("\n" + "="*50)
print("Best Hyperparameters:")
print("="*50)
for key, value in best_config.items():
    print(f"{key}: {value}")
print(f"\nBest CV Loss: {best_trial.value:.4f}")

# Save to exp_folder (same structure as your original code)
exp_name = best_config.get('experiment_name', 'default')
exp_folder = f"configs/{exp_name}"
os.makedirs(exp_folder, exist_ok=True)

# Save best config (same path as your original code)
config_path = f"{exp_folder}/best_config.json"
with open(config_path, "w") as f:
    json.dump(best_config, f, indent=4)

print(f"\nConfig saved to: {config_path}")

# Save metadata (same structure as your original code)
metadata = {
    'best_loss': best_trial.value,
    'best_std': best_trial.user_attrs.get('std', None),
    'num_trials': len(study.trials),
    'num_completed': len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]),
    'num_pruned': len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]),
    'num_failed': len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL]),
    'timestamp': pd.Timestamp.now().isoformat()
}
metadata_path = f"{exp_folder}/metadata.json"
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Metadata saved to: {metadata_path}")

# Save study results to ray_results directory
study_results_path = f"{results_dir}/study.pkl"
import joblib
joblib.dump(study, study_results_path)

# Save trials dataframe
trials_df = study.trials_dataframe()
trials_df.to_csv(f"{results_dir}/trials.csv", index=False)

print(f"\nStudy results saved to: {results_dir}/")
print("  - study.pkl (complete study object)")
print("  - trials.csv (all trial results)")

# Optional: Generate visualizations
try:
    import plotly
    
    fig1 = optuna.visualization.plot_optimization_history(study)
    fig1.write_html(f"{exp_folder}/optimization_history.html")
    
    fig2 = optuna.visualization.plot_param_importances(study)
    fig2.write_html(f"{exp_folder}/param_importances.html")
    
    fig3 = optuna.visualization.plot_parallel_coordinate(study)
    fig3.write_html(f"{exp_folder}/parallel_coordinate.html")
    
    print(f"\nVisualizations saved to {exp_folder}/")
    
except ImportError:
    print("\nInstall plotly for visualizations: pip install plotly kaleido")

print("\n" + "="*50)
print("TUNING COMPLETE")
print("="*50)

C:\Users\Jojie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\_experimental.py:31: ExperimentalWarning:

Argument ``multivariate`` is an experimental feature. The interface can change in the future.

[I 2026-02-11 22:47:59,925] A new study created in memory with name: gru_tuning


Using CPU only
STARTING HYPERPARAMETER TUNING
Total trials: 100
Concurrent trials: 4
Device: cpu
Study name: gru_tuning
Storage: ray_results/ (Optuna equivalent)



  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Jojie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\distributions.py:524: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64] which is of type list.

C:\Users\Jojie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\distributions.py:524: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64, 64] which is of type list.

C:\Users\Jojie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\distributions.py:524: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for pers

Trial 0 failed: ray_train() got an unexpected keyword argument 'trial'
[I 2026-02-11 22:47:59,949] Trial 0 finished with value: inf and parameters: {'num_channels': [256, 256], 'kernel_size': 3, 'dropout': 0.18012931121174408, 'lr': 0.0027922133130790848, 'wd': 1.056526508740203e-05, 'factor': 0.4246376248898365, 'patience': 10, 'batch_size': 32, 'seq_len': 18, 'l1_lambda': 0.007027019122754872}. Best is trial 0 with value: inf.


Best trial: 0. Best value: inf:   0%|          | 0/100 [00:00<?, ?it/s]

...

[I 2026-02-11 22:48:28,236] Trial 99 finished with value: inf and parameters: {'num_channels': [128], 'kernel_size': 3, 'dropout': 0.15799491185309084, 'lr': 0.0011839865028727922, 'wd': 5.288421426915471e-06, 'factor': 0.2904766852451249, 'patience': 5, 'batch_size': 16, 'seq_len': 24, 'l1_lambda': 0.0005054031705634077}. Best is trial 0 with value: inf.


Best trial: 0. Best value: inf: 100%|██████████| 100/100 [00:29<00:00,  3.42it/s]
[W 2026-02-11 22:48:29,296] Trial 0 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,303] Trial 1 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,304] Trial 2 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,305] Trial 3 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,305] Trial 4 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,306] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,306] Trial 6 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,306] Trial 7 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,307] Trial 8 is omitted in visualizatio


Progress: 100/100 trials | Best loss: inf
Trial    State        Loss         Std          hidden_size  lr           batch_size  
----------------------------------------------------------------------------------------------------


Best Hyperparameters:
num_channels: [256, 256]
kernel_size: 3
dropout: 0.18012931121174408
lr: 0.0027922133130790848
wd: 1.056526508740203e-05
factor: 0.4246376248898365
patience: 10
batch_size: 32
seq_len: 18
l1_lambda: 0.007027019122754872
lag_periods: [1, 3, 12]
features: ['BIR', 'BOC', 'Other Offices', 'TotalTrade_PHPMN', 'NominalGDP_disagg', 'Pop_disagg']
labels: ['BIR', 'BOC', 'Other Offices']
dummy_vars: ['COVID-19', 'TRAIN', 'CREATE', 'FIST', 'BIR_COMM']
experiment_name: test

Best CV Loss: inf

Config saved to: configs/test/best_config.json
Metadata saved to: configs/test/metadata.json

Study results saved to: c:\Users\Jojie\Desktop\School\Masters\Sem 2\01 Capstone\TCN\ray_results\gru_tuning/
  - study.pkl (complete study object)
  - trials.csv (al

[W 2026-02-11 22:48:29,335] Trial 61 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,336] Trial 62 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,337] Trial 63 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,339] Trial 64 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,339] Trial 65 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,340] Trial 66 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,340] Trial 67 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,340] Trial 68 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,341] Trial 69 is omitted in visualization because its objective value is inf or nan.
[W 2026-02-11 22:48:29,341] 


Visualizations saved to configs/test/

TUNING COMPLETE


In [36]:
exp_folder = "configs/test"

config_path = f"{exp_folder}/best_config.json"

with open(config_path, "r") as f:
     best_config = json.load(f)
# Create args with best config
args = train.Arguments(
    **best_config,
    seed=1,
    epoch=200,  # Train longer for final model
)

# Set device
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
args.device = device
args.use_lags = True
args.lag_periods = [1, 3]
# Load dataset
dataset = train.load_dataset(args)

train.set_seed(args.seed)
exp_name = args.experiment_name if hasattr(args, 'experiment_name') else 'final'

# Split cv_data into train/val
train_size = int(0.85 * len(dataset['cv_data']))
final_train_data = dataset['cv_data'][:train_size]
final_train_labels = dataset['cv_labels'][:train_size]
final_val_data = dataset['cv_data'][train_size:]
final_val_labels = dataset['cv_labels'][train_size:]

# Scale data
exp_name = args.experiment_name if hasattr(args, 'experiment_name') else 'final'

# Scale data with organized folder structure
final_train_scaled = transform_data(final_train_data, f"Transforms/{exp_name}/train_scaled.pkl")
final_train_labels_scaled = transform_data(final_train_labels, f"Transforms/{exp_name}/labels_scaled.pkl")
final_val_scaled = transform_data(final_val_data, f"Transforms/{exp_name}/train_scaled.pkl")
final_val_labels_scaled = transform_data(final_val_labels, f"Transforms/{exp_name}/labels_scaled.pkl")
final_test_scaled = transform_data(dataset['test_data'], f"Transforms/{exp_name}/test_scaled.pkl")
final_test_labels_scaled = transform_data(dataset['test_labels'], f"Transforms/{exp_name}/labels_scaled.pkl")

# Create datasets
train_dataset = TimeSeriesDataset(final_train_scaled, final_train_labels_scaled, seq_len=args.seq_len)
val_dataset = TimeSeriesDataset(final_val_scaled, final_val_labels_scaled, seq_len=args.seq_len)
test_dataset = TimeSeriesDataset(final_test_scaled, final_test_labels_scaled, seq_len=args.seq_len)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [44]:
# Create final model
train.set_seed(args.seed)
final_model = TCNRegressor(
    input_size=dataset['input_size'],  
    num_channels=args.num_channels,
    output_size=dataset['output_size'],  
    kernel_size=args.kernel_size,
    dropout=args.dropout
).to(device)

args.train_criterion = nn.HuberLoss(delta=1.0)
#args.train_criterion = nn.MSELoss()
args.test_criterion = train.MAPELoss()
args.final = True
# Train and evaluate
test_loss, test_preds, train_losses, val_losses = train.run(
    final_model,
    train_loader,
    val_loader,
    test_loader,
    args,
    fold=None
)

print(f"\n{'='*50}")
print(f"Final Model Results:")
print(f"{'='*50}")
print(f"Final Test Loss (MAPE): {test_loss:.4f}%")

C:\Users\Jojie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning:

`torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.



RuntimeError: Given groups=1, weight of size [256, 23, 3], expected input[32, 18, 23] to have 23 channels, but got 18 channels instead

In [ ]:
# Debug: Check actual input shape
print("="*50)
print("Dataset Info:")
print("="*50)
print(f"Reported input_size: {dataset['input_size']}")
print(f"Reported output_size: {dataset['output_size']}")

# Get actual shape from dataloader
for batch_x, batch_y in train_loader:
    print(f"\nActual batch input shape: {batch_x.shape}")
    print(f"Actual batch output shape: {batch_y.shape}")
    actual_input_size = batch_x.shape[1]  # Number of channels
    print(f"Actual input channels: {actual_input_size}")
    print(f"Actual output channels: {batch_y.shape[1]}")
    break

# Create final model with actual input size
train.set_seed(args.seed)
final_model = TCNRegressor(
    input_size=actual_input_size,  # Use actual shape from data
    num_channels=args.num_channels,
    output_size=batch_y.shape[1],  # Use actual output channels
    kernel_size=args.kernel_size,
    dropout=args.dropout
).to(device)

print(f"\n{'='*50}")
print(f"Model instantiated with:")
print(f"  input_size={actual_input_size}")
print(f"  num_channels={args.num_channels}")
print(f"  output_size={batch_y.shape[1]}")
print(f"  kernel_size={args.kernel_size}")
print(f"  dropout={args.dropout}")
print(f"{'='*50}\n")

args.train_criterion = nn.HuberLoss(delta=1.0)
args.test_criterion = train.MAPELoss()
args.final = True

# Train and evaluate
test_loss, test_preds, train_losses, val_losses = train.run(
    final_model,
    train_loader,
    val_loader,
    test_loader,
    args,
    fold=None
)

print(f"\n{'='*50}")
print(f"Final Model Results:")
print(f"{'='*50}")
print(f"Final Test Loss (MAPE): {test_loss:.4f}%")

Dataset Info:
Reported input_size: 23
Reported output_size: 3

Actual batch input shape: torch.Size([32, 18, 23])
Actual batch output shape: torch.Size([32, 3])
Actual input channels: 18
Actual output channels: 3

Model instantiated with:
  input_size=18
  num_channels=[256, 256]
  output_size=3
  kernel_size=3
  dropout=0.18012931121174408



C:\Users\Jojie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning:

`torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.



Epoch 1/200 - Train Loss: 82.4182, Val Loss: 0.1186
Epoch 100/200 - Train Loss: 1.2167, Val Loss: 0.0369
Epoch 200/200 - Train Loss: 1.2058, Val Loss: 0.0353

Final Model Results:
Final Test Loss (MAPE): 26.6850%


In [45]:
# DEBUG: Track which channels/features are included
import os

base_dir = os.path.dirname(os.path.abspath(train.__file__))
data_dir = os.path.join(base_dir, "Data")

# Load raw data
btr_data = pd.read_csv(os.path.join(data_dir, "cordata.csv")).set_index("Date")
macro_data = pd.read_csv(os.path.join(data_dir, "disaggregated.csv")).fillna(0)
macro_data = macro_data.rename(columns={'Unnamed: 3': 'Date'}).set_index('Date')
dummy = pd.read_csv(os.path.join(data_dir, "dummy.csv")).fillna(0).set_index("Date")

# Check what's available
print("="*80)
print("CHANNEL AVAILABILITY ANALYSIS")
print("="*80)

features = args.features if hasattr(args, 'features') else []
labels = args.labels if hasattr(args, 'labels') else []
dummy_vars = args.dummy_vars if hasattr(args, 'dummy_vars') else []

print(f"\nFeatures requested: {features}")
print(f"Features found in data: {[f for f in features if f in btr_data.columns or f in macro_data.columns]}")
print(f"Features MISSING: {[f for f in features if f not in btr_data.columns and f not in macro_data.columns]}")

print(f"\nLabels requested: {labels}")
print(f"Labels found: {[l for l in labels if l in btr_data.columns]}")

print(f"\nDummy vars requested: {dummy_vars}")
print(f"Dummy vars available in dummy.csv: {list(dummy.columns)}")
print(f"Dummy vars FOUND in data: {[d for d in dummy_vars if d in dummy.columns]}")
print(f"Dummy vars MISSING: {[d for d in dummy_vars if d not in dummy.columns]}")

# Calculate expected channels
lag_count = len(args.lag_periods) * len(labels)
main_features_count = len([f for f in features if f in btr_data.columns or f in macro_data.columns])
dummy_count = len([d for d in dummy_vars if d in dummy.columns])
seasonal_features = ['month_sin', 'month_cos', 'quarter_sin', 'quarter_cos', 'is_tax_season', 'is_year_end']
seasonal_count = len(seasonal_features)

print("\n" + "="*80)
print("CHANNEL COUNT BREAKDOWN:")
print("="*80)
print(f"Main features: {main_features_count}")
print(f"Lag features (labels × lags): {lag_count} ({len(labels)} labels × {len(args.lag_periods)} lags)")
print(f"Dummy variables: {dummy_count}")
print(f"Seasonal features: {seasonal_count}")
print(f"{'─'*80}")
print(f"EXPECTED total: {main_features_count + lag_count + dummy_count + seasonal_count}")
print(f"ACTUAL (from data): {dataset['input_size']}")
print(f"DIFFERENCE: {main_features_count + lag_count + dummy_count + seasonal_count - dataset['input_size']}")
print("="*80)

CHANNEL AVAILABILITY ANALYSIS

Features requested: ['BIR', 'BOC', 'Other Offices', 'TotalTrade_PHPMN', 'NominalGDP_disagg', 'Pop_disagg']
Features found in data: ['BIR', 'BOC', 'Other Offices', 'TotalTrade_PHPMN', 'NominalGDP_disagg', 'Pop_disagg']
Features MISSING: []

Labels requested: ['BIR', 'BOC', 'Other Offices']
Labels found: ['BIR', 'BOC', 'Other Offices']

Dummy vars requested: ['COVID-19', 'TRAIN', 'CREATE', 'FIST', 'BIR_COMM']
Dummy vars available in dummy.csv: ['COVID-19', 'TRAIN', 'CREATE', 'FIST', 'BIR_COMM', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20']
Dummy vars FOUND in data: ['COVID-19', 'TRAIN', 'CREATE', 'FIST', 'BIR_COMM']
Dummy vars MISSING: []

CHANNEL COUNT BREAKDOWN:
Main features: 6
Lag features (labels × lags): 6 (3 labels × 2 lags)
Dummy variables: 5
Seasonal features: 6
──────────────

In [48]:
batch_x

tensor([[[0.0807, 0.0347, 0.0601,  ..., 0.0000, 1.0000, 0.0000],
         [0.0021, 0.0126, 0.0559,  ..., 0.0000, 1.0000, 0.0000],
         [0.0155, 0.0428, 0.0250,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0389, 0.0904, 0.1077,  ..., 0.5000, 0.0000, 0.0000],
         [0.0325, 0.0836, 0.0934,  ..., 0.5000, 0.0000, 0.0000],
         [0.0127, 0.0768, 0.1220,  ..., 0.5000, 0.0000, 0.0000]],

        [[0.0021, 0.0126, 0.0559,  ..., 0.0000, 1.0000, 0.0000],
         [0.0155, 0.0428, 0.0250,  ..., 0.0000, 0.0000, 0.0000],
         [0.0466, 0.0311, 0.0684,  ..., 0.5000, 0.0000, 0.0000],
         ...,
         [0.0325, 0.0836, 0.0934,  ..., 0.5000, 0.0000, 0.0000],
         [0.0127, 0.0768, 0.1220,  ..., 0.5000, 0.0000, 0.0000],
         [0.0471, 0.0913, 0.0637,  ..., 1.0000, 0.0000, 0.0000]],

        [[0.0155, 0.0428, 0.0250,  ..., 0.0000, 0.0000, 0.0000],
         [0.0466, 0.0311, 0.0684,  ..., 0.5000, 0.0000, 0.0000],
         [0.0217, 0.0259, 0.0999,  ..., 0.5000, 0.0000, 0.